<a href="https://colab.research.google.com/github/201524495/2021_FSRnT/blob/main/Save_Load.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q pyyaml h5py  # HDF5 포맷으로 모델을 저장하기 위해서 필요합니다

In [ ]:
import os

import tensorflow as tf
from tensorflow import keras
#from pathlib import Path

print(tf.version.VERSION)

2.4.1


In [ ]:
# 예제 데이터 셋 받기
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

In [ ]:
# 간단한 Sequential 모델을 정의

def create_model():
  model = tf.keras.models.Sequential([
          keras.layers.Dense(512, activation = 'relu', input_shape = (784,)),
          keras.layers.Dropout(0.2),
          keras.layers.Dense(10)
  ])

  model.compile(optimizer = 'adam',
                loss = tf.losses.SparseCategoricalCrossentropy(from_logits = True),
                metrics = ['accuracy'])
  
  return model

  # 모델 객체를 만듭니다.
model = create_model()

  # 모델 구조를 출력
model.summary()

Model: "sequential_78"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_156 (Dense)            (None, 512)               401920    
_________________________________________________________________
dropout_78 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_157 (Dense)            (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# 훈련하는 동안 체크포인트 저장
# 체크포인트 콜백 사용하기

# create folder
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# 모델의 가중치를 저장하는 콜백 만들기
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath = checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose = 1)

# 새로운 콜백으로 모델 훈련
model.fit(train_images, 
          train_labels,  
          epochs=10,
          validation_data=(test_images,test_labels),
          callbacks=[cp_callback]) # 콜백을 훈련에 전달

# 옵티마이저는 상태를 저장하는 것과 관련되어 경고가 발생할 수 있다.
# 이 경고는 이전 사용 방식을 권장하지 않기 위함이며 무시해도 상관 없다. 

Epoch 1/10
32/32 [==============================] - 1s 12ms/step - loss: 1.6175 - accuracy: 0.5131 - val_loss: 0.7095 - val_accuracy: 0.7880

Epoch 00001: saving model to training_1/cp.ckpt
Epoch 2/10
32/32 [==============================] - 0s 7ms/step - loss: 0.4451 - accuracy: 0.8747 - val_loss: 0.5615 - val_accuracy: 0.8180

Epoch 00002: saving model to training_1/cp.ckpt
Epoch 3/10
32/32 [==============================] - 0s 7ms/step - loss: 0.3134 - accuracy: 0.9191 - val_loss: 0.4663 - val_accuracy: 0.8500

Epoch 00003: saving model to training_1/cp.ckpt
Epoch 4/10
32/32 [==============================] - 0s 7ms/step - loss: 0.1915 - accuracy: 0.9606 - val_loss: 0.4479 - val_accuracy: 0.8520

Epoch 00004: saving model to training_1/cp.ckpt
Epoch 5/10
32/32 [==============================] - 0s 7ms/step - loss: 0.1452 - accuracy: 0.9710 - val_loss: 0.4177 - val_accuracy: 0.8630

Epoch 00005: saving model to training_1/cp.ckpt
Epoch 6/10
32/32 [==============================] - 0s

In [ ]:
ls {checkpoint_dir}

checkpoint  cp.ckpt.data-00000-of-00001  cp.ckpt.index


In [ ]:
# 기본 모델 객체를 만듦
model = create_model()

# 모델 평가
loss, acc = model.evaluate(test_images, test_labels, verbose = 2)
print("훈련되지 않은 모델의 정확도 : {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 2.4379 - accuracy: 0.0530
훈련되지 않은 모델의 정확도 :  5.30%


In [ ]:
# 가중치 로드
model.load_weights(checkpoint_path)

# 모델 재평가
loss, acc = model.evaluate(test_images, test_labels, verbose = 2)
print("복원된 모델의 정확도 : {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 0.4149 - accuracy: 0.8680
복원된 모델의 정확도 : 86.80%


In [ ]:
# 체크포인트 콜백 매개변수

# 파일 이름에 에포크 번호를 포함시킵니다(`str.format` 포맷)
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# 다섯 번째 에포크마다 가중치를 저장하기 위한 콜백을 만듭니다
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    period=5)

# 새로운 모델 객체를 만듭니다
model = create_model()

# `checkpoint_path` 포맷을 사용하는 가중치를 저장합니다
model.save_weights(checkpoint_path.format(epoch=0))

# 새로운 콜백을 사용하여 모델을 훈련합니다
model.fit(train_images, 
          train_labels,
          epochs=50, 
          callbacks=[cp_callback],
          validation_data=(test_images,test_labels),
          verbose=0)


Epoch 00005: saving model to training_2/cp-0005.ckpt

Epoch 00010: saving model to training_2/cp-0010.ckpt

Epoch 00015: saving model to training_2/cp-0015.ckpt

Epoch 00020: saving model to training_2/cp-0020.ckpt

Epoch 00025: saving model to training_2/cp-0025.ckpt

Epoch 00030: saving model to training_2/cp-0030.ckpt

Epoch 00035: saving model to training_2/cp-0035.ckpt

Epoch 00040: saving model to training_2/cp-0040.ckpt

Epoch 00045: saving model to training_2/cp-0045.ckpt

Epoch 00050: saving model to training_2/cp-0050.ckpt


In [ ]:
ls {checkpoint_dir}

checkpoint                        cp-0025.ckpt.index
cp-0000.ckpt.data-00000-of-00001  cp-0030.ckpt.data-00000-of-00001
cp-0000.ckpt.index                cp-0030.ckpt.index
cp-0005.ckpt.data-00000-of-00001  cp-0035.ckpt.data-00000-of-00001
cp-0005.ckpt.index                cp-0035.ckpt.index
cp-0010.ckpt.data-00000-of-00001  cp-0040.ckpt.data-00000-of-00001
cp-0010.ckpt.index                cp-0040.ckpt.index
cp-0015.ckpt.data-00000-of-00001  cp-0045.ckpt.data-00000-of-00001
cp-0015.ckpt.index                cp-0045.ckpt.index
cp-0020.ckpt.data-00000-of-00001  cp-0050.ckpt.data-00000-of-00001
cp-0020.ckpt.index                cp-0050.ckpt.index
cp-0025.ckpt.data-00000-of-00001


In [ ]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2/cp-0050.ckpt'

In [ ]:
# 새로운 모델 객체를 제작
model = create_model()

# 이전에 저장한 가중치를 로드합니다
model.load_weights(latest)

# 모델을 재평가합니다
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("복원된 모델의 정확도: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 0.4946 - accuracy: 0.8750
복원된 모델의 정확도: 87.50%


In [ ]:
# 수동으로 가중치 저장하기

# 가중치를 저장합니다
model.save_weights('./checkpoints/my_checkpoint')

# 새로운 모델 객체를 만듭니다
model = create_model()

# 가중치를 복원합니다
model.load_weights('./checkpoints/my_checkpoint')

# 모델을 평가합니다
loss,acc = model.evaluate(test_images,  test_labels, verbose=2)
print("복원된 모델의 정확도: {:5.2f}%".format(100*acc))

32/32 - 0s - loss: 0.4946 - accuracy: 0.8750
복원된 모델의 정확도: 87.50%


In [ ]:
# 전체 모델 저장
#새로운 모델 객체를 만들고 훈련
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# SavedModel로 전체 모델을 저장합니다
!mkdir -p saved_model
model.save('saved_model/my_model')

Epoch 1/5
32/32 [==============================] - 0s 4ms/step - loss: 1.5629 - accuracy: 0.5395
Epoch 2/5
32/32 [==============================] - 0s 4ms/step - loss: 0.4300 - accuracy: 0.8892
Epoch 3/5
32/32 [==============================] - 0s 4ms/step - loss: 0.2790 - accuracy: 0.9367
Epoch 4/5
32/32 [==============================] - 0s 4ms/step - loss: 0.1894 - accuracy: 0.9565
Epoch 5/5
32/32 [==============================] - 0s 4ms/step - loss: 0.1318 - accuracy: 0.9831
INFO:tensorflow:Assets written to: saved_model/my_model/assets


In [ ]:
# my_model 디렉토리
ls saved_model

SyntaxError: ignored

In [ ]:
# assets 폴더, saved_model.pb, variables 폴더
ls saved_model/my_model/

In [ ]:
new_model = tf.keras.models.load_model('saved_model/my_model')

# 모델 구조 확인
new_model.summary

In [ ]:
# 복원된 모델 평가
loss, acc = new_model.evaluate(test_images, test_labels, verbose = 2)
print("복원된 모델의 정확도: {:5.2f}%".format(100*acc))

print(new_model.predict(test_images).shape)

In [ ]:
# HDF5 파일로 저장
# 새로운 모델 객체를 만들고 훈련
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# 전체 모델을 HDF5로 저장
# 
model.save('my_model.h5')

In [ ]:
# 가중치와 옵티마이저를 포함하여 정확히 동일한 모델을 다시 생성
new_model = tf.keras.models.load_model('my_model.h5')

#모델 구조 출력
new_model.summary()

In [ ]:
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print("복원된 모델의 정확도: {:5.2f}%".format(100*acc))